<a href="https://colab.research.google.com/github/dream80/DeepFaceLab_Colab/blob/master/DeepFaceLab_Colab_zh_cn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 简介

无需多说，一路运行即可！


# 第一步  把workspace上传到谷歌云盘


1. 登录谷歌云，在根目录创建一个目录 ***DeepFaceLab*** 
2. 把本地 ***workspace.zip*** 上传到 ***DeepFaceLab***目录  
3. 把谷歌云盘挂载到Colab
4. 用CD命令进入DeepFaceLab目录。



In [0]:
#挂载谷歌云盘
#点击链接授权，复制授权码，填入框框。

from google.colab import drive

drive.mount('/content/drive', force_remount=True)

In [0]:
#进入目录 
%cd /content/drive/My Drive/DeepFaceLab/

In [0]:
# 第一次使用，可以直接git clone一个workspace
!git clone https://github.com/dream80/DFLWorkspace.git workspace

In [0]:
# 当然，正常情况肯定是上传自己的workspace，上传前压缩一下，然后用下面的命令解压。
# 注意要放到DeepFaceLab目录下。
#!unzip workspace.zip



#第二步 安装DeepFaceLab


In [0]:
# 获取DFL源代码v1.0

!git clone -b v1.0 https://github.com/dream80/DeepFaceLab_Colab.git

#!git clone https://github.com/dream80/DeepFaceLab_Colab.git

In [0]:
# 进入DeepFaceLab_Colab目录
%cd /content/drive/My Drive/DeepFaceLab/DeepFaceLab_Colab

#获取最新源代码
#!git pull


In [0]:
# 安装Python依赖
!pip install -r requirements_colab.txt
!pip install --upgrade scikit-image

# Step 3. Extract faces

In [0]:
# 确保路径正确，进入DeepFaceLab_Colab

%cd /content/drive/My Drive/DeepFaceLab/DeepFaceLab_Colab

In [0]:
#Dst视频转图片

!python main.py videoed extract-video --input-file ../workspace/data_dst.mp4 --output-dir ../workspace/data_dst/

In [0]:
#Dst提取脸部图片

!python main.py extract --input-dir ../workspace/data_dst --output-dir ../workspace/data_dst/aligned --detector s3fd --debug-dir ../workspace/data_dst/aligned_debug

In [0]:
# 排序，可以通过谷歌云盘查看，删除不良图片

!python main.py sort --input-dir ../workspace/data_dst/aligned --by hist

In [0]:
#Src 视频转图片

!python main.py videoed extract-video --input-file ../workspace/data_src.mp4 --output-dir ../workspace/data_src/

In [0]:
#Src 提取脸部图片

!python main.py extract --input-dir ../workspace/data_src --output-dir ../workspace/data_src/aligned --detector s3fd --debug-dir ../workspace/data_src/aligned_debug

In [0]:
#排序，可以通过谷歌云盘查看，删除不良图片

!python main.py sort --input-dir ../workspace/data_src/aligned --by hist

# 第四步. 训练模型


*  支持H128,SAE等模型
*  不想训练了可以点击停止，停止时会抛出异常，但是没什么关系。下次可以继续训练
*  如果想要查看history ，先停止训练，然后点击下面第三段代码

In [0]:
# Running trainer  H128

!python main.py train --training-data-src-dir ../workspace/data_src/aligned --training-data-dst-dir ../workspace/data_dst/aligned --model-dir ../workspace/model --model H128 --no-preview

In [0]:
# Running trainer. SAE 

!python main.py train --training-data-src-dir ../workspace/data_src/aligned --training-data-dst-dir ../workspace/data_dst/aligned --model-dir ../workspace/model --model SAE --no-preview

In [0]:
#@title
# 训练效果预览，先把训练暂停，然后允许这一段代码。
# 如果不想暂停训练，你可以点击这个链接。 https://github.com/dream80/DeepFaceLab_Colab/blob/master/ViewLastHistory_H128.ipynb
# 在新的页面上查看。

import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import os
from google.colab import files
 
print(tf.__version__)

imgpath="/content/drive/My Drive/DeepFaceLab/workspace/model/H128_history/"
tlist=os.listdir(imgpath)
tlist.sort(key=lambda x:int(x[:-5]))
lastpic=tlist[-1]

image_raw = tf.gfile.GFile(imgpath+lastpic,'rb').read()   #bytes
img = tf.image.decode_jpeg(image_raw)  #Tensor

plt.rcParams['figure.figsize'] = (8, 4.0)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray' 
plt.rcParams['savefig.dpi'] =200 
plt.rcParams['figure.dpi'] = 200



with tf.Session() as sess:
    plt.imshow(img.eval())
 

# 第五部. 转换

In [0]:
#用src中的脸替换dst的脸 
!python main.py convert --input-dir ../workspace/data_dst --output-dir ../workspace/data_dst/merged --aligned-dir ../workspace/data_dst/aligned --model-dir ../workspace/model --model H128

In [0]:
#把替换好的图片转换成视频

!python main.py videoed video-from-sequence --input-dir ../workspace/data_dst/merged --output-file ../workspace/result.mp4 --reference-file ../workspace/data_dst.mp4

# 其他

上面每一部产生的结果，可以在谷歌云盘上进行预览。

https://drive.google.com/drive/my-drive

如果有问题，可以联系： wpgdream@gmail.com